### tool定义

In [2]:
from tool import Calculator, Weather, CurrentDay
calculator = Calculator()
weather = Weather()
current_day = CurrentDay()

### llm 定义

In [3]:
from langchain.chat_models import ChatOpenAI
import os
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", openai_api_key=os.environ['OPENAI_API_KEY'])


In [4]:
llm.invoke("你好")

AIMessage(content='你好！有什么可以帮助你的吗？')

### 大模型直接回答

In [5]:
query = "去年营收13,242.445,241.43元, 今年营收25,432,543,569.42元, 同比增长了多少？"
resp = llm.invoke(query)
print(resp)

content='去年营收为13,242,445,241.43元，今年营收为25,432,543,569.42元。\n\n同比增长率 = (今年营收 - 去年营收) / 去年营收 × 100%\n= (25,432,543,569.42 - 13,242,445,241.43) / 13,242,445,241.43 × 100%\n= 12,190,098,327.99 / 13,242,445,241.43 × 100%\n≈ 92%\n\n所以，今年营收同比增长了大约92%。'


In [6]:
calculator.invoke("25,432,543,569.42/13,242,445,241.43")

1.92053228129291

### 构造agent

In [7]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(
        [calculator, weather], llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )
agent

AgentExecutor(verbose=True, tags=['structured-chat-zero-shot-react-description'], agent=StructuredChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math, args: {{\'para\': {{\'title\': \'Para\', \'type\': \'string\'}}}}\nWeather: Get the Temperature of given day, args: {{\'day\': {{\'title\': \'Day\', \'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or Calculator, Weather\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer

In [8]:
# query="上海和北京哪个城市好？"
resp = agent.run(query)
# resp = agent.run("去年营收13242445245, 今年营收25432543569, 同比增长了多少？")
resp



> Entering new AgentExecutor chain...
I can help you calculate the year-over-year growth in revenue. To do that, I will use the Calculator tool to find the percentage increase in revenue from last year to this year.

Action:
```{
  "action": "Calculator",
  "action_input": "((25432543569.42 - 13242445241.43) / 13242445241.43) * 100"
}```


Observation: 92.053228129291
Thought:I have calculated the year-over-year revenue growth to be approximately 92.05%.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The year-over-year revenue growth is approximately 92.05%."
}
```

> Finished chain.


'The year-over-year revenue growth is approximately 92.05%.'